In [62]:
import pickle
import json
import numpy as np

In [42]:
class Model_prediction:
    
    def __init__(self, model):
        self._model = model
        
        
    def __json_to_dict(self):
        with open('json/description.json', 'r') as f:
            self.di_description = json.load(f)
        with open('json/precaution.json', 'r') as f:
            self.di_precaution = json.load(f)
        with open('json/disease_eng_to_ru.json', 'r') as f:
            self.disease_eng_to_ru = json.load(f)
        with open('json/descr_eng_to_ru.json', 'r') as f:
            self.descr_eng_to_ru = json.load(f)
        with open('json/precs_eng_to_ru.json', 'r') as f:
            self.precs_eng_to_ru = json.load(f)
            
    
    def prediction(self, data: list) -> json:
        self.__json_to_dict()
        pred = self._model.predict([data])[0]
        descr = self.di_description[pred]
        precaution = self.di_precaution[pred]
        result = {'Disease':self.disease_eng_to_ru[pred], 
                  'Description':self.descr_eng_to_ru[descr], 
                  'Precaution':[self.precs_eng_to_ru[prec] for prec in precaution]}
        return json.dumps(result, indent=4, ensure_ascii=False)

In [75]:
with open('json/severity_weights.json', 'r') as f:
    severity_weights = json.load(f)

In [76]:
def symps_to_labels(symps):
    arr = np.array([severity_weights[symp] for symp in symps])
    return np.pad(arr, (0, 17-len(arr))).tolist()

In [77]:
model1 = pickle.load(open('model/model1_rf.pkl','rb'))

In [78]:
symps = ['skin_rash','shivering','spotting_urination','itching',
         'skin_rash','nodal_skin_eruptions','itching','itching']

In [79]:
sympotms = symps_to_labels(symps)

In [86]:
model = Model_prediction(model1)
pred = model.prediction(sympotms)
pred

'{\n    "Disease": "Псориаз",\n    "Description": "Псориаз — это распространенное кожное заболевание, при котором образуются толстые, красные, неровные пятна, покрытые серебристыми чешуйками. Они могут появиться где угодно, но чаще всего появляются на коже головы, локтях, коленях и нижней части спины. Псориаз не передается от человека к человеку. Иногда это случается у членов одной семьи.",\n    "Precaution": [\n        "Мыть руки теплой водой с мылом",\n        "Остановить кровотечение с помощью давления",\n        "Проконсультируйтесь со специализированным врачом",\n        "Соляные ванны"\n    ]\n}'

In [85]:
json.loads(pred)

{'Disease': 'Псориаз',
 'Description': 'Псориаз — это распространенное кожное заболевание, при котором образуются толстые, красные, неровные пятна, покрытые серебристыми чешуйками. Они могут появиться где угодно, но чаще всего появляются на коже головы, локтях, коленях и нижней части спины. Псориаз не передается от человека к человеку. Иногда это случается у членов одной семьи.',
 'Precaution': ['Мыть руки теплой водой с мылом',
  'Остановить кровотечение с помощью давления',
  'Проконсультируйтесь со специализированным врачом',
  'Соляные ванны']}